In [ ]:
import tensorflow as tf
import numpy as np
import time
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
#make figures bigger on HiDPI monitors
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
import scipy.io as io

tf.random.set_seed(42)

def tfp_function_factory(model, *args):
    """A factory to create a function required by tfp.optimizer.lbfgs_minimize.
    Based on the example from https://pychao.com/2019/11/02/optimize-tensorflow-keras-models-with-l-bfgs-from-tensorflow-probability/
    Args:
        model [in]: an instance of `tf.keras.Model` or its subclasses.
        *args: arguments to be passed to model.get_grads method        
        
    Returns:
        A function that has a signature of:
            loss_value, gradients = f(model_parameters).
    """

    # obtain the shapes of all trainable parameters in the model
    shapes = tf.shape_n(model.trainable_variables)
    n_tensors = len(shapes)

    # we'll use tf.dynamic_stitch and tf.dynamic_partition later, so we need to
    # prepare required information first
    count = 0
    idx = [] # stitch indices
    part = [] # partition indices

    for i, shape in enumerate(shapes):
        n = np.prod(shape)
        idx.append(tf.reshape(tf.range(count, count+n, dtype=tf.int32), shape))
        part.extend([i]*n)
        count += n

    part = tf.constant(part)
    @tf.function
    def assign_new_model_parameters(params_1d):
        """A function updating the model's parameters with a 1D tf.Tensor.

        Args:
            params_1d [in]: a 1D tf.Tensor representing the model's trainable parameters.
        """

        params = tf.dynamic_partition(params_1d, part, n_tensors)
        for i, (shape, param) in enumerate(zip(shapes, params)):
            model.trainable_variables[i].assign(tf.reshape(param, shape))

    # now create a function that will be returned by this factory
    @tf.function
    def f(params_1d):
        """A function that can be used by tfp.optimizer.lbfgs_minimize.

        This function is created by function_factory.

        Args:
           params_1d [in]: a 1D tf.Tensor.

        Returns:
            A scalar loss and the gradients w.r.t. the `params_1d`.
        """
        assign_new_model_parameters(params_1d)        
        loss_value, grads = model.get_grad(*args)
        grads = tf.dynamic_stitch(idx, grads)

        # print out iteration & loss
        f.iter.assign_add(1)
        if f.iter%model.print_epoch == 0:
            tf.print("Iter:", f.iter, "loss:", loss_value)

        return loss_value, grads

    # store these information as members so we can use them outside the scope
    f.iter = tf.Variable(0)
    f.idx = idx
    f.part = part
    f.shapes = shapes
    f.assign_new_model_parameters = assign_new_model_parameters

    return f

class CalculateUPhi4th_DD(tf.keras.Model): 
    def __init__(self, layers_list, train_op, num_epoch, print_epoch,model_data,data_type):
        super().__init__()
        self.model_layers_list = layers_list
        self.train_op = train_op
        self.num_epoch = num_epoch
        self.print_epoch = print_epoch
        self.data_type = data_type
        self.l = model_data['l']
        self.B = model_data['B']
        self.Gc = model_data['Gc']
        self.E = model_data['E']
        self.crack_loc = model_data['crack_loc']

    #@tf.function                                
    def call(self, X_all):
        output =[]
        for i in range(len(X_all)):
            u_val, phi_val = self.u_phi(X_all[i][:,0:1],i)
            output.append(tf.concat([u_val,phi_val],1))
        return output 
    
    def dirichletBound(self, X, xPhys, net_indx):
        u_val = (xPhys+1)*(xPhys-1)*X[:, 0:1]
        return u_val

   
    def net_hist(self,x):        
        init_hist = tf.zeros_like(x)
        dist = tf.abs(x-self.crack_loc)
        init_hist = tf.where(dist < 0.5*self.l, self.B*self.Gc*0.5*(1-(2*dist/self.l))/self.l, init_hist)        
        return init_hist  
    
    def net_energy(self,x, net_indx):

        u, phi = self.u_phi(x, net_indx)        
        hist = self.net_hist(x)
        
        g = (1-phi)**2
        u_x, phi_x = self.du_phi(x, net_indx)
        nabla = phi_x**2
        _, phi_xx = self.d2u_phi(x, net_indx)
        laplacian = phi_xx**2        
        sigmaX = self.E*u_x 
        
        energy_u = 0.5*g*sigmaX*u_x
        energy_phi = 0.5*self.Gc * (phi**2/self.l + self.l*nabla + 0.5*self.l**3*laplacian) + g*hist        
        return energy_u, energy_phi

    # Running the model
    #@tf.function
    def u_phi(self,X, net_indx):
        xPhys = X
        X = 2.0*(X - self.bounds["lb"][net_indx])/(self.bounds["ub"][net_indx] - self.bounds["lb"][net_indx]) - 1.0
        for l in self.model_layers_list[net_indx]:
            X = l(X)
        u = self.dirichletBound(X, xPhys, net_indx)
        phi = X[:, 1:2]
        return u, phi
                
    # Return the first derivatives
    def du_phi(self, xPhys, net_indx):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(xPhys)
            u_val,phi_val = self.u_phi(xPhys, net_indx)
        du_val = tape.gradient(u_val, xPhys)
        dphi_val = tape.gradient(phi_val, xPhys)
        return du_val, dphi_val
    
    # Return the second derivative
    def d2u_phi(self, xPhys, net_indx):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(xPhys)
            du_val, dphi_val = self.du_phi(xPhys, net_indx)
        d2u_val = tape.gradient(du_val, xPhys)
        d2phi_val = tape.gradient(dphi_val, xPhys)
        return d2u_val,d2phi_val
        
    #Custom loss function
    @tf.function
    def get_all_losses(self, Xint_all, Wint_all, Yint_all,
                       Xif_all, Yif_all, if_list):
        # calculate the losses for energy, phi and external work
        loss_energy_u = 0.
        loss_energy_phi = 0.
        loss_ext_work = 0. 
        for i in range(len(Xint_all)):
            xPhys_int = Xint_all[i][:,0:1]
            yPhys_int = Yint_all[i][:,0:1]
            u, _ = self.u_phi(xPhys_int,i)
            energy_u, energy_phi = self.net_energy(xPhys_int,i)
            loss_energy_u += tf.reduce_sum(energy_u*Wint_all[i])
            loss_energy_phi += tf.reduce_sum(energy_phi*Wint_all[i])
            loss_ext_work += tf.reduce_sum(u*yPhys_int*Wint_all[i])  

        # compute the interface loss and interior loss along the interface
        iface_loss = 0.
        for i in range(len(Xif_all)):
            indx_a = if_list[i][0]
            indx_b = if_list[i][1]
            xPhys = Xif_all[i][:, 0:1]                                                  
            u_a, phi_a = self.u_phi(xPhys, indx_a)
            u_b, phi_b = self.u_phi(xPhys, indx_b)
            dudx_a, dphidx_a = self.du_phi(xPhys, indx_a)
            dudx_b, dphidx_b = self.du_phi(xPhys, indx_b)
            iface_loss += tf.reduce_mean(tf.math.square(u_a-u_b))
            iface_loss += tf.reduce_mean(tf.math.square(phi_a-phi_b))
            iface_loss += tf.reduce_mean(tf.math.square(dudx_a-dudx_b))
            iface_loss += tf.reduce_mean(tf.math.square(dphidx_a-dphidx_b))

        return loss_energy_u, loss_energy_phi, loss_ext_work, iface_loss
    
    @tf.function
    def get_loss(self,Xint_all, Wint_all,Yint_all, Xif_all, Yif_all, if_list):
        loss_energy_u, loss_energy_phi, loss_ext_work, iface_loss = self.get_all_losses(Xint_all, Wint_all,
                                                Yint_all, Xif_all, Yif_all, if_list)
        return loss_energy_u + loss_energy_phi + iface_loss - loss_ext_work
          
    # get gradients
    @tf.function
    def get_grad(self,Xint_all, Wint_all,Yint_all, Xif_all, Yif_all, if_list):
        with tf.GradientTape() as tape:
            tape.watch(self.trainable_variables)
            L = self.get_loss(Xint_all, Wint_all,Yint_all, Xif_all, Yif_all, if_list)
        g = tape.gradient(L, self.trainable_variables)
        return L, g    
      
    # perform gradient descent
    def network_learn(self, Xint_all, Wint_all,Yint_all, Xif_all, Yif_all, if_list):
        self.bounds = {"lb": [], "ub": []}
        for i in range(len(Xint_all)):
            xmin = tf.math.reduce_min(Xint_all[i][:,0])
            xmax = tf.math.reduce_max(Xint_all[i][:,0])
            self.bounds["lb"].append(xmin)
            self.bounds["ub"].append(xmax)
        for i in range(self.num_epoch):
            L, g = self.get_grad(Xint_all, Wint_all,Yint_all, Xif_all, Yif_all, if_list)
            self.train_op.apply_gradients(zip(g, self.trainable_variables))
            if i%self.print_epoch==0:
                loss_energy_u, loss_energy_phi, loss_ext_work, iface_loss = self.get_all_losses(Xint_all, 
                                Wint_all,Yint_all, Xif_all, Yif_all, if_list)
                L = loss_energy_u + loss_energy_phi + iface_loss - loss_ext_work
                print("Epoch {} loss: {}, loss_energy_u: {}, loss_energy_phi: {}, iface_loss: {}".format(i, 
                                        L, loss_energy_u, loss_energy_phi,iface_loss))  
                    
def loading_fun(x):
 
    y = np.sin(np.pi*x)
    return y

def exact_sol(x, l):
    
    u_exact = np.sin(np.pi*x)/(np.pi)**2 + np.where(x < 0.0,
                                    -1.0*(1+x)/np.pi, (1-x)/np.pi)
    phi_exact = np.exp(-np.absolute(x-0.)/l)*(1+np.absolute(x-0.)/l)
    
    return u_exact, phi_exact


xmin = -1.
xmid = 0.
xmax = 1.
dom1 = np.array([-1.0, 0])
dom2 = np.array([0, 1.0])
   

data_type = "float64"
data = io.loadmat('coordinates.mat')

sys.exit()
# plt.scatter(Xint, np.zeros_like(Xint), s=0.5, color='black')

model_data = dict()
model_data['E'] = 1.
model_data['l'] = 0.0125
model_data['B'] = 1000.
model_data['Gc'] = 2.7
model_data['crack_loc'] = 0.

#define the model 
tf.keras.backend.set_floatx(data_type)
l1 = tf.keras.layers.Dense(10, "swish")
l12 = tf.keras.layers.Dense(10, "swish")
l13 = tf.keras.layers.Dense(10, "swish")
l1_out = tf.keras.layers.Dense(2, None)

l2 = tf.keras.layers.Dense(10, "swish")
l22 = tf.keras.layers.Dense(10, "swish")
l23 = tf.keras.layers.Dense(10, "swish")
l2_out = tf.keras.layers.Dense(2, None)

train_op = tf.keras.optimizers.Adam()
num_epoch = 20000
print_epoch = 100
pred_model = CalculateUPhi4th_DD([[l1, l12, l13, l1_out], [l2, l22, l23, l2_out]], train_op,
                                      num_epoch, print_epoch, model_data, data_type)                              

#convert the training data to tensors
Xint1_tf = tf.convert_to_tensor(data['Xint_1'])
Xint2_tf = tf.convert_to_tensor(data['Xint_2'])

Wint1_tf = tf.convert_to_tensor(data['wgtsPhys1'].astype(data_type))
Wint2_tf = tf.convert_to_tensor(data['wgtsPhys2'].astype(data_type))

Yint1_tf = tf.convert_to_tensor(data['Yint_1'])
Yint2_tf = tf.convert_to_tensor(data['Yint_2'])


Xint_all = [Xint1_tf, Xint2_tf]
Wint_all = [Wint1_tf, Wint2_tf]
Yint_all = [Yint1_tf, Yint2_tf]

Xif_12 = np.array([[0]]).astype(data_type)
if_list = [(0,1)]
Yif_12 = loading_fun(Xif_12).astype(data_type)

Xif_12 = [tf.convert_to_tensor(Xif_12)]
Yif_12 = [tf.convert_to_tensor(Yif_12)]

t0 = time.time()
print("Training (ADAM)...")

pred_model.network_learn(Xint_all, Wint_all, Yint_all, Xif_12, Yif_12, if_list)
t1 = time.time()
print("Time taken (ADAM)", t1-t0, "seconds")

print("Training (TFP-BFGS)...")

loss_func = tfp_function_factory(pred_model, Xint_all, Wint_all, Yint_all, Xif_12, Yif_12, if_list)
# convert initial model parameters to a 1D tf.Tensor
init_params = tf.dynamic_stitch(loss_func.idx, pred_model.trainable_variables)
# train the model with BFGS solver
results = tfp.optimizer.bfgs_minimize(
value_and_gradients_function=loss_func, initial_position=init_params,
      max_iterations=1000, tolerance=1e-14)
# after training, the final optimized parameters are still in results.position
# so we have to manually put them back to the model
loss_func.assign_new_model_parameters(results.position)
t2 = time.time()
print("Time taken (BFGS)", t2-t1, "seconds")
print("Time taken (all)", t2-t0, "seconds")


print("Testing...")
numPtsTest = 300
x1_test = np.linspace(xmin, xmid, numPtsTest).astype(data_type)
x1_test = np.array(x1_test)[np.newaxis].T
x1_tf = tf.convert_to_tensor(x1_test)

x2_test = np.linspace(xmid, xmax, numPtsTest).astype(data_type)
x2_test = np.array(x2_test)[np.newaxis].T
x2_tf = tf.convert_to_tensor(x2_test)

XTest_all = [x1_tf, x2_tf]
YTest_all = pred_model(XTest_all)

# splitting the u and phi components 
UTest_all = []
PTest_all = []
for i in range(len(YTest_all)):
    UTest_all.append(YTest_all[i][:, 0:1])
    PTest_all.append(YTest_all[i][:, 1:2])

# u_exact, phi_exact = exact_sol(x_test, model_data['l'])

err_all_u = []
err_all_phi = []
err_norm_u  = 0.
err_norm_phi = 0.
exact_norm_u = 0.
exact_norm_phi = 0.
for i in range(len(YTest_all)):
    u_exact, phi_exact = exact_sol(XTest_all[i][:,0:1], model_data['l'])
    u_test = UTest_all[i]
    phi_test = PTest_all[i]
    err_all_u.append(u_exact - u_test)
    err_all_phi.append(phi_exact - phi_test)
    err_norm_u += np.sum((u_exact - u_test)**2)
    err_norm_phi += np.sum((phi_exact - phi_test)**2)
    exact_norm_u += np.sum(u_exact**2)
    exact_norm_phi += np.sum(phi_exact**2)
rel_err_u = np.sqrt(err_norm_u/exact_norm_u)
rel_err_phi = np.sqrt(err_norm_phi/exact_norm_phi)
print("Relative error u: ", rel_err_u)
print("Relative error phi: ", rel_err_phi)

plt.plot(x1_test, UTest_all[0], x2_test, UTest_all[1], label = 'u_pred')
plt.plot(x1_test, PTest_all[0], x2_test, PTest_all[1], label = 'phi_pred')

x12_test = np.concatenate((x1_test, x2_test), axis=0)
u_exact, phi_exact = exact_sol(x12_test, model_data['l'])

plt.plot(x12_test, u_exact, '--', label = 'u_exact')
plt.plot(x12_test, phi_exact, '--', label = 'phi_exact')

plt.legend()
plt.show()
